<a href="https://colab.research.google.com/github/sumeetku/test/blob/master/deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pylab inline
from google.colab import drive

drive.mount('/content/gdrive')


Populating the interactive namespace from numpy and matplotlib
Mounted at /content/gdrive


In [3]:
from fastai.vision import *
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [4]:
cd /content/gdrive/MyDrive/Colab\ Notebooks/

/content/gdrive/MyDrive/Colab Notebooks


In [5]:
!ls

 bbc_train			    predict-volcanic-eruptions.ipynb
 Copy_of_cv_assign.ipynb	   'Project data'
 export.pkl			    Untitled0.ipynb
 kaggle.json			    Untitled1.ipynb
 license_plate_reader.ipynb	    Untitled2.ipynb
 modelling.ipynb		    Untitled3.ipynb
 predict-volcanic-eruptions	    Untitled4.ipynb
 predict_volcanic_eruptions.ipynb   Untitled5.ipynb


In [6]:
!ls bbc_train/

bboxes.json		   export.pkl  images  sample_student.py
classification_labels.csv  handin.zip  masks


In [7]:
path=Path('bbc_train')

In [8]:
path.ls()

[PosixPath('bbc_train/.DS_Store'),
 PosixPath('bbc_train/bboxes.json'),
 PosixPath('bbc_train/classification_labels.csv'),
 PosixPath('bbc_train/masks'),
 PosixPath('bbc_train/images'),
 PosixPath('bbc_train/.ipynb_checkpoints'),
 PosixPath('bbc_train/sample_student.py'),
 PosixPath('bbc_train/export.pkl'),
 PosixPath('bbc_train/handin.zip')]

In [9]:
labels=pd.read_csv(path/'classification_labels.csv')
labels.head()

,filename,label,label_index
0,P2050936.jpg,ball,2
1,P2050922.jpg,ball,2
2,P2060163.jpg,brick,1
3,P2060003.jpg,ball,2
4,P2060017.jpg,ball,2


In [56]:
def get_y_fn(x): return path/'masks'/(x.stem + '.png')
classes = array(['background', 'brick', 'ball', 'cylinder'])

src = (SegmentationItemList.from_folder(path/'images')
       .split_by_rand_pct(0.2)
       .label_from_func(get_y_fn, classes=classes))

data = (src.transform(get_transforms(), tfm_y=True, size=(256,256))
        .databunch(bs=16)
        .normalize(imagenet_stats))


In [57]:
def conv_trans(ni, nf, ks = 4, stride = 2, padding = 1):
    return nn.Sequential(
        nn.ConvTranspose2d(ni, nf, kernel_size=ks, bias=False, stride=stride, padding = padding), 
        nn.ReLU(inplace = True), 
        nn.BatchNorm2d(nf))

In [58]:
custom_head=nn.Sequential(conv_trans(512, 256), 
                          conv_trans(256, 128),
                          conv_trans(128, 64),
                          conv_trans(64, 32), 
                          nn.ConvTranspose2d(32, 4, kernel_size=4, bias=False, stride=2, padding = 1))

In [59]:


def pixel_accuracy(yhat, y):
    y_=y.squeeze(dim=1)
    yhat_=yhat.argmax(dim=1)
    return (y_==yhat_).sum().float()/y.numel()



In [67]:
learn=cnn_learner(data, models.resnet18, custom_head=custom_head, metrics=pixel_accuracy)

In [85]:
lr=1e-2
learn.fit_one_cycle(10, slice(lr/10, lr))


epoch,train_loss,valid_loss,pixel_accuracy,time
0,0.022133,0.022635,0.990853,00:17
1,0.023603,0.028029,0.989438,00:17
2,0.026687,0.038231,0.985117,00:17
3,0.029783,0.040898,0.984407,00:17
4,0.028691,0.037212,0.986058,00:18
5,0.025618,0.024635,0.989932,00:17
6,0.023392,0.021208,0.991466,00:17
7,0.021634,0.021064,0.991236,00:17
8,0.020527,0.024168,0.990910,00:17
9,0.019754,0.024387,0.991067,00:17


In [33]:
!pwd

/content/gdrive/My Drive/Colab Notebooks


In [86]:
learn.export('export.pkl')

In [87]:
!cp bbc_train/images/export.pkl .


In [88]:
## ---------------------------- ##
##
## eval.py
## Deep learning programming challenge evaluation code. 
## 
## ---------------------------- ##

import numpy as np
import time
from fastai.vision import *
from pathlib import Path



def evaluate(data_path='./bbc_train',
             model_dir = '',
             batch_size=64, 
             im_size=(256,256)):

    
    print("1. Loading Data...")

    path=Path(data_path)
    def get_y_fn(x): return path/'masks'/(x.stem + '.png')
    classes = array(['background', 'brick', 'ball', 'cylinder'])

    src = (SegmentationItemList.from_folder(path/'images')
       .split_by_rand_pct(0.0)
       .label_from_func(get_y_fn, classes=classes))

    #Don't normalize data here - assume normalization happens inside of Model. 
    data = src.transform(get_transforms(), tfm_y=True, size=im_size).databunch(bs=batch_size)
    print(data)


    print("2. Instantiating Model...")
    M = Model(path=model_dir, file='export.pkl')


    print("3. Evaluating on Test Images...")
    x,y = data.one_batch()
    #Extract class label from mask:
    class_labels=np.array([np.unique(y[i][y[i]!=0])[0] for i in range(x.shape[0])])

    #Extract bounding box from mask:
    bboxes=np.zeros((x.shape[0], 4))
    for i in range(x.shape[0]):
        rows,cols= np.where(y[i, 0]!=0)
        bboxes[i, :] = np.array([rows.min(), cols.min(), rows.max(), cols.max()])

    preds=M.predict(x)
    
    classification_accuracy=(np.array([data.classes[i] for i in class_labels])==np.array(preds[0])).sum()/len(preds[0])
    bbox_score = 1 - np.mean(np.abs(bboxes-preds[1]))/(x.shape[-1]) #Divide by image height to rougly normalize score
    segmentation_accuracy=float((preds[2] == np.array(y.squeeze(1))).sum())/y.numel()
    combined_accuracy=np.mean([classification_accuracy, bbox_score, segmentation_accuracy])

    return combined_accuracy, classification_accuracy, bbox_score, segmentation_accuracy


def calculate_score(combined_accuracy):
    if combined_accuracy >= 0.99: score = 10
    elif combined_accuracy >= 0.98: score = 9
    elif combined_accuracy >= 0.97: score = 8
    elif combined_accuracy >= 0.96: score = 7
    elif combined_accuracy >= 0.95: score = 6
    elif combined_accuracy >= 0.90: score = 5
    else: score = 4
    return score


if __name__ == '__main__':
    program_start = time.time()
    combined_accuracy, classification_accuracy, bbox_score, segmentation_accuracy = evaluate()
    score = calculate_score(combined_accuracy)

    program_end = time.time()
    total_time = round(program_end - program_start, 3)
    
    print("\nDone!")
    print("Execution time (seconds) = ", total_time)
    print("Score = ", score, 
         "\nCombined accuracy = ", combined_accuracy, 
         "\nClassification accuracy = ", classification_accuracy,
         "\nbbox score = ", bbox_score,
         "\nSegmentation Accuracy = ", segmentation_accuracy)



1. Loading Data...
ImageDataBunch;

Train: LabelList (942 items)
x: SegmentationItemList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: SegmentationLabelList
ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256),ImageSegment (1, 256, 256)
Path: bbc_train/images;

Valid: LabelList (0 items)
x: SegmentationItemList

y: SegmentationLabelList

Path: bbc_train/images;

Test: None
2. Instantiating Model...
3. Evaluating on Test Images...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Done!
Execution time (seconds) =  4.742
Score =  10 
Combined accuracy =  0.9942366282145182 
Classification accuracy =  1.0 
bbox score =  0.990386962890625 
Segmentation Accuracy =  0.9923229217529297


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
